In [2]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
[ Info: Precompiling MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83]


In [3]:
pwd()

"/Users/ivanspirandelli/Doktor/Code/MorphoMol/MorphoMolNotebooks"

In [4]:
folder = "../../Data/hpc_out/2_6r7m_oj_0_0_os_0_85/"
folder = "../../Data/hpc_out/rwm_wp_2_6r7m/"

id_index = 1
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, "_")[id_index]) ? parse(Int, split(file, "_")[id_index]) : max
    end
end

min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$folder$file" in_out_data
        id = parse(Int, split(file, "_")[id_index])
        string = "$(id): $(in_out_data.input.T) | $(in_out_data.input.ε) | $(in_out_data.output.αs[end]) | $(in_out_data.output.Es[1]), $(minimum(in_out_data.output.Es)), $(in_out_data.output.Es[end])"
        evaluation_strings[id] = string
        min_Es[id] = minimum(in_out_data.output.Es)
    end
end

LoadError: KeyError: key "in_out_data" not found

In [9]:
argmin(min_Es)

57

In [8]:
for elem in evaluation_strings
    println(elem)
end

1: 2.0 | 0.0 | 0.055647623 | 8219.578304409264, 8167.725301025351, 8175.7986357888
2: 2.0 | 0.0 | 0.0273336 | 8219.578304345747, 8164.513421792593, 8173.876727574058
3: 2.0 | 0.0 | 0.03167717 | 8219.57830401968, 8168.826707196028, 8175.976690357623
4: 2.0 | 0.0 | 0.03839247 | 8219.578304407985, 8157.620558777056, 8162.728408181958
5: 2.0 | 0.0 | 0.007483615 | 8342.603229685159, 8168.496170583278, 8170.717095042941
6: 2.0 | 0.0 | 0.0108670015 | 8219.578304128916, 8173.718714734241, 8181.717020263267
7: 2.0 | 0.0 | 0.0068299286 | 8219.578303918304, 8172.882398886147, 8177.901304432319
8: 2.0 | 0.0 | 0.019939596 | 8219.578303980428, 8150.112391187765, 8150.428475705565
9: 2.0 | 0.0 | 0.013546041 | 8219.578303957402, 8162.317371740355, 8162.317371740355
10: 2.0 | 0.0 | 0.009289576 | 8207.8099251459, 8157.643098217686, 8159.273390004637
11: 2.0 | 0.0 | 0.014425842 | 8216.50130661928, 8169.778638136508, 8170.842245211545
12: 2.0 | 0.0 | 0.05547243 | 8219.578303949487, 8164.394661938923, 8164

In [10]:
for file in readdir(folder)
    try
        @load "$folder$file" in_out_data
        mindex = argmin(in_out_data.output.Es)
        state = in_out_data.output.states[mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([in_out_data.input.template_radii for _ in 1:in_out_data.input.number_of_molecules]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, in_out_data.input.template_mol), radii, "$(folder)$(id)", in_out_data.input.number_of_molecules, length(in_out_data.input.template_radii))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [6]:
@load "assets/hpc_out/2_6r7m_touching_oj_0_0_os_0_85/2_6r7m_touching_13_protor_oj_0_0_os_0_85.jld2" in_out_data

1-element Vector{Symbol}:
 :in_out_data

In [13]:
mindex = argmin(in_out_data.output.Es)
lower = 500
upper = 100
out_folder = "assets/output/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([in_out_data.input.template_radii for _ in 1:in_out_data.input.number_of_molecules]))
for (i,j) in enumerate(mindex-lower:mindex+upper)
    state = in_out_data.output.states[j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, in_out_data.input.template_mol), radii, "$(out_folder)$(i)", in_out_data.input.number_of_molecules, length(in_out_data.input.template_radii))
end


# Compare to Experimental Structures

In [24]:
@load "assets/hpc_out/3_6r7m_assembled_oj_0_0_os_0_85/3_6r7m_assembled_47_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [33]:
@load "assets/hpc_out/3_6r7m_assembled_oj_0_0_os_0_85/3_6r7m_assembled_47_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))


R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.992567  0.121696  0.000000; -0.121696  0.992567  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-19.48193, 22.01644, -46.53000]

state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end


permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(MorphoMol.Utilities.sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
    println()
end

1.2873006221221273, 18.550779293943858, 22.747256593704485
14.19511216992349

1.3774676242284738, 18.550779293943858, 22.656875406631716
14.195040774934682

1.2873006221221273, 1.3775764058075557, 2.8189008939530753
1.8279259739609195

18.550888075522934, 1.3775764058075557, 22.656875406631716
14.195113295987403

1.3774676242284738, 1.2874090345163622, 2.8189008939530753
1.8279258508993037

18.550888075522934, 1.2874090345163622, 22.747256593704485
14.195184567914595



In [35]:
@load "assets/hpc_out/3_6r7m_stacked_oj_0_0_os_0_85/3_6r7m_stacked_28_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))

R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.803441  0.595384  0.000000; -0.595384  0.803441  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-67.99970, 135.02619, -25.38000]


state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end

permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(MorphoMol.Utilities.sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
    println()
end

1.0801358636489278, 1.761615083090717, 2.5839573080281175
1.8085694182559209

3.9681361891428946, 1.761615083090717, 7.226195653808025
4.318648975347212

1.0801358636489278, 1.9392750832172398, 6.0393168036705065
3.019575916845558

5.427780645913811, 1.9392750832172398, 7.226195653808025
4.864417127646359

3.9681361891428946, 5.184605884136937, 6.0393168036705065
5.064019625650112

5.427780645913811, 5.184605884136937, 2.5839573080281175
4.398781279359622



In [38]:
@load "assets/hpc_out/3_6r7m_assembled_oj_0_0_os_0_85/3_6r7m_assembled_47_protor_oj_0_0_os_0_85.jld2" in_out_data
mindex = argmin(in_out_data.output.Es)
in_out_data.output.Vs[mindex], in_out_data.output.As[mindex], in_out_data.output.Ms[mindex], in_out_data.output.Xs[mindex]

(84123.59f0, 20349.809f0, -5968.993f0, 188.49556f0)

In [36]:
@load "assets/hpc_out/3_6r7m_stacked_oj_0_0_os_0_85/3_6r7m_stacked_28_protor_oj_0_0_os_0_85.jld2" in_out_data
mindex = argmin(in_out_data.output.Es)
in_out_data.output.Vs[mindex], in_out_data.output.As[mindex], in_out_data.output.Ms[mindex], in_out_data.output.Xs[mindex]

(85262.81f0, 21122.312f0, -6398.94f0, 62.831852f0)

# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334


In [ ]:
println("1EI7")